In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from sklearn import set_config
set_config(display="diagram")

In [ ]:
from sklearn.datasets import load_digits
from sklearn.svm import SVC
import time
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score, accuracy_score, roc_auc_score, make_scorer
from sklearn.metrics import confusion_matrix, classification_report, precision_recall_curve
from sklearn.metrics import make_scorer
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split

In [ ]:
#loading the dataset
data = pd.read_csv("data/bank_full_raw.csv", sep = ';')

In [ ]:
#Display of column name and data type
data.info()

In [ ]:
data.count()

In [ ]:
data.head(10)

In [ ]:
#Removal of rows & columns with NaN/Null values and duplicates
data.dropna()
data.drop_duplicates()
data.count()

In [ ]:
df = data
df = df.replace('yes','1')
df = df.replace('no','0')
df[['default', 'loan', 'housing', 'y']] = df[['default', 'loan', 'housing', 'y']].astype(int)

In [ ]:
df.info()

In [ ]:
print(df[df.columns[df.isna().any()]].sum())

In [ ]:
df.groupby("month").count()

In [ ]:
data.groupby("job").count()

In [ ]:
data.groupby("marital").count()

In [ ]:
data.groupby("education").count()

In [ ]:
data.groupby("housing").count()

In [ ]:
data.groupby("loan").count()

In [ ]:
data.groupby("contact").count()

In [ ]:
data.groupby("campaign").count()

In [ ]:
data.groupby("poutcome").count()

In [ ]:
data.groupby("y").count()

In [ ]:
df[df["poutcome"] == 'success'].filter(df[df["y"] == 'yes'])

In [ ]:
df[df["poutcome"] == 'unknown'].filter(df[df["y"] == 'yes'])

In [ ]:
df[df["poutcome"] == 'failure'].filter(df[df["y"] == 'yes'])

In [ ]:
sns.set(style="ticks")
sns.pairplot(df)
plt.show()

In [ ]:
f, ax = plt.subplots(figsize = (15,8)) 
sns.barplot(data=df, x="job", y = "y", palette="winter_r")
sns.despine(left = True, bottom = True)
ax.set(xlabel='job', ylabel='y')

In [ ]:
f, ax = plt.subplots(figsize = (15,8)) 
sns.barplot(x="marital", y = "y", data = df, palette="winter_r")
sns.despine(left = True, bottom = True)
ax.set(xlabel='marital', ylabel='y')

In [ ]:
f, ax = plt.subplots(figsize = (15,8)) 
sns.barplot(x="education", y = "y", data = df, palette="winter_r")
sns.despine(left = True, bottom = True)
ax.set(xlabel='education', ylabel='y')

In [ ]:
colors = sns.color_palette('pastel')[0:5]
f, ax = plt.subplots(figsize = (15,8)) 
sns.barplot(x="housing", y = "y", data = df, palette="winter_r")
sns.despine(left = True, bottom = True)
ax.set(xlabel='housing', ylabel='y')

### Further exploration

In [ ]:
f, ax = plt.subplots(figsize = (15,8)) 
sns.countplot(x="y", data=df, palette="winter_r")
sns.despine(left = True, bottom = True)
plt.show()

In [ ]:
f, ax = plt.subplots(figsize = (15,8)) 
sns.countplot(x="loan", data=df, palette="winter_r")
sns.despine(left = True, bottom = True)
plt.show()

In [ ]:
countNoLoan = len(df[df.y == 0])
countHaveLoan = len(df[df.y == 1])
print("\nPercentage of Customer can't have a loan: {:.2f}%".format((countNoLoan / (len(df.y))*100)))
print("Percentage of Customer can have a loan: {:.2f}%\n".format((countHaveLoan / (len(df.y))*100)))

In [ ]:
df.groupby('y').mean()

In [ ]:
colors = sns.color_palette('winter_r')[0:5]
pd.crosstab(df.age,df.y).plot(kind="bar",figsize=(20,6), color = colors)
plt.title('Subscribed to Term Deposit based on Age')
plt.xlabel('Age')
plt.ylabel('Subsctibed')
plt.show()

From age 25-60, consumers tend to subscribe to the term deposit campaign reqest.

In [ ]:
colors = sns.color_palette('winter_r')[0:5]
pd.crosstab(df.housing, df.y).plot(kind="bar", figsize=(15,6), color=colors)
plt.title('client subscribed to term deposit who do or not have Housing loans')
plt.xlabel('housing ("no" = No Housing Loan , "yes" = Have housing loan)')
plt.xticks(rotation=0)
plt.legend(["No Housing Loan", "Have housing loan"])
plt.ylabel('Subscribed')
plt.show()

In [ ]:
f, ax = plt.subplots(figsize = (15,8)) 
plt.scatter(x=df.job[df.y== 1], y=df.campaign[(df.y== 1)], c="magenta")
plt.scatter(x=df.job[df.y== 0], y=df.campaign[(df.y== 0)])
plt.title('Client contacted based on their education')
plt.legend(["Contacted", "Not Contacted"])
plt.xlabel("Job")
plt.ylabel("Campaign")
plt.show()

In [ ]:
f, ax = plt.subplots(figsize = (15,8)) 
plt.scatter(x=df.marital[df.y== 1], y=df.campaign[(df.y== 1)], c="magenta")
plt.scatter(x=df.marital[df.y== 0], y=df.campaign[(df.y== 0)])
plt.title('client contacted based on their marital status')
plt.legend(["Contacted", "Not Contacted"])
plt.xlabel("Marital")
plt.ylabel("Campaign")
plt.show()

In [ ]:
f, ax = plt.subplots(figsize = (15,8)) 
plt.scatter(x=df.job[df.y== 1], y=df.pdays[(df.y== 1)], c="magenta")
plt.scatter(x=df.job[df.y== 0], y=df.pdays[(df.y== 0)])
plt.title('# of Days passed from the last contct with clients based on job')
plt.legend(["Contacted", "Not Contacted"])
plt.xlabel("Job")
plt.ylabel("# of Days passed")
plt.show()

In [ ]:
f, ax = plt.subplots(figsize = (15,8)) 
plt.scatter(x=df.age[df.y== 1], y=df.pdays[(df.y== 1)], c="magenta")
plt.scatter(x=df.age[df.y== 0], y=df.pdays[(df.y== 0)])
plt.title('# of Days passed from the last contct with clients based on age')
plt.legend(["Contacted", "Not Contacted"])
plt.xlabel("Age")
plt.ylabel("# of Days passed")
plt.show()

In [ ]:
f, ax = plt.subplots(figsize = (15,8)) 
plt.scatter(x=df.age[df.y== 1], y=df.previous[(df.y== 1)], c="magenta")
plt.scatter(x=df.age[df.y== 0], y=df.previous[(df.y== 0)])
plt.title('# of contct performed before this campaign with clients based on age')
plt.legend(["Contacted", "Not Contacted"])
plt.xlabel("Age")
plt.ylabel("# of Days passed")
plt.show()

In [ ]:
f, ax = plt.subplots(figsize = (15,8)) 
plt.scatter(x=df.poutcome[df.y== 1], y=df.previous[(df.y== 1)], c="magenta")
plt.scatter(x=df.poutcome[df.y== 0], y=df.previous[(df.y== 0)])
plt.title('Outcome of previous campaign')
plt.legend(["Subsctibed", "Not Subscribed"])
plt.xlabel("Outcome")
plt.ylabel("Previous Subscription Status")
plt.show()

In [ ]:
f, ax = plt.subplots(figsize = (15,8)) 
plt.scatter(x=df.contact[df.y== 1], y=df.previous[(df.y== 1)], c="magenta")
plt.scatter(x=df.contact[df.y== 0], y=df.previous[(df.y== 0)])
plt.title('Contact method during previous campaign')
plt.legend(["Subsctibed", "Not Subscribed"])
plt.xlabel("Contact Method")
plt.ylabel("Previous Subscription Status")
plt.show()

In [ ]:
# writing the curated copy of the data
df.to_csv('data/bank_curated_data.csv', sep=';')

In [ ]:
df